In [ ]:
%run SequenceDeltaAlgo.ipynb

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import string
import pandas as pd
import re
from datetime import datetime
from datetime import timedelta

import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt

# Constants ------------------------------------------------------------------------------
# fed url
DAY_FS='[YYYYMMDD]'
URL_FS= 'https://www.federalreserve.gov/newsevents/pressreleases/monetary[YYYYMMDD]a.htm'
FROM_DATE = datetime(2020, 1, 29, 0, 0, 1)
global lastFSD


# Methods scraping and feed ---------------------------------------------------------------
def nextBusinessDay(d):
    current_date = d
    while True:
        current_date += timedelta(days=1)
        weekday = current_date.weekday()
        if weekday >= 5:
            continue
        else:
            return current_date
        



def strLoadArticle(strdate):
    datedArticle = 'fedStates/'+ strdate
    if os.path.exists(datedArticle):
        return datedArticle
    else:
        datedUrl = URL_FS.replace(DAY_FS, strdate) 
        try:
            page = requests.get(datedUrl)
            soup = BeautifulSoup(page.content, 'html.parser')
            divs =  soup.find(id="article").findAll('div')
            ldivs = list(map(lambda d: len(d.findChildren("p", recursive=False)), divs))
            articleDiv = divs[ldivs.index(max(ldivs))]
            ps = articleDiv.findAll('p')
            
            if ps[-1].text.startswith('Implementation Note issued') :
                for tag in articleDiv.findAll(True):
                    if len(tag.attrs)> 0 :
                        tag.attrs = {}
                content = list(map(lambda p: str(p), ps))
                with open(datedArticle, "w") as f:
                    sad = str(" <br> ".join(content))
                    sad += createSimpleNPL(sad, datedUrl, strdate)
                    f.write(sad)
                return datedArticle 
            else:
                return None
        except:
            return None
        

def loadArticle(d):
    strdate = d.strftime("%Y%m%d")
    return strLoadArticle(strdate)
  
    
def extractArticle(sdate):
    datedArticle = strLoadArticle(sdate)
    if not datedArticle is None:
        with open(datedArticle, 'r') as f:
             content = f.read()
        return content
    else:
        return ""
        
def splitArticle(a):
    a = a.replace('<p>', '')
    a = a.replace('</p>', '')
    np = a.rfind('<div id="snpl"')
    content = filter(lambda w: w.strip() != '', a[0:np].split(' ')) 
    wordProxies = list(map(lambda w: WordProxy(w), content))
    score =  a[np:]
    return wordProxies, score 
 
    
def startupArticles():
    global lastFSD
    dates = []
    dateRegex = re.compile(r'(\d\d\d\d\d\d\d\d)')

    for datedFile in os.listdir('fedStates/'):
        if dateRegex.search(datedFile):
            dates.append(datedFile)
    fromDate = FROM_DATE
    if dates:
        try:
            dates.sort(reverse=True)
            fromDate = datetime.strptime(dates[0], '%Y%m%d')
            lastFSD = dates[0]
        except:
            fromDate = FROM_DATE
    
    dnow = datetime.now()
    while fromDate <= dnow :
        if not loadArticle(fromDate) is None:
            lastFSD = fromDate.strftime("%Y%m%d") 
        fromDate = nextBusinessDay(fromDate)

        
def dailyCheckArticle():
    global lastFSD
    sdnow = datetime.now().strftime("%Y%m%d")
    if lastFSD < sdnow:
        if not strLoadArticle(sdnow) is None:
            lastFSD = sdnow
            generateNPLGraph()
            return lastFSD
    return None

def calculateNPL(blob):
    raw = BeautifulSoup(blob).get_text()
    tokens = nltk.word_tokenize(raw)
    tokens_without_sw = [word for word in tokens if not word in stopwords.words() and word not in string.punctuation]
    sia = SentimentIntensityAnalyzer()
    score = sia.polarity_scores(' '.join(tokens_without_sw))
    return score

def createSimpleNPL(sad, url, d): 
    nplBloc = """
                <br>
                <div id="snpl" style="background-color:#FCF3CF;"> 
                    <h3>Sentiment Analysis for Fed Statment date: <a href='[url]'>[date]</h3>
                    <h4><span style="color:#008000;">Positive  : </span>[+]</h4>
                    <h4><span style="color:#FF0000;">Negative  : </span>[-]</h4>
                    <h4><span style="color:#808080;">Neutre    : </span>[=]</h4>
                    <h4><span style="color:#000000;">Compouned : </span>[x]</h4>
                  
                 </div>
              """
    nplBloc = nplBloc.replace('[url]', url)
    nplBloc = nplBloc.replace('[date]', d)
    
    pscore = calculateNPL(sad)
    nplBloc = nplBloc.replace('[+]', str(pscore["pos"]))
    nplBloc = nplBloc.replace('[-]', str(pscore["neg"]))
    nplBloc = nplBloc.replace('[=]', str(pscore["neu"]))
    nplBloc = nplBloc.replace('[x]', str(pscore["compound"]))
    
    return nplBloc

def parseSimpleNPL(nplBloc):
    strScore = []
    for h4Bloc in nplBloc.split('</h4>')[:-1] :
        strScore.append(h4Bloc.split('</span>')[1]) 
    return strScore

def createDeltaNPL(fix, new, old, sda0, sda1, nplScore0, nplScore1): 
    dNplBloc = """
                <br>
                <div id="deltaNpl" style="background-color:#FCF3CF;"> 
                    <h3>Sentiment Analysis for Fed Statment Delta:</h3>
                    <table style="font-size:20px;width: 70%;">
                        <tr style="background-color:#DCDCDC;">
                          <th style="text-align: left;width: 19%;border-bottom: thin solid;"></th>
                          <th style="text-align: left;width: 18%;border-bottom: thin solid;"><span style="background-color:#ff6424;">[d0]</span></th>
                          <th style="text-align: left;width: 18%;border-bottom: thin solid;"><span style="background-color:lightgreen">[d1]</span></th>
                          <th style="text-align: left;width: 15%;border-bottom: thin solid;">Fixed Branch</th>
                          <th style="text-align: left;width: 15%;border-bottom: thin solid;color:red">Old Branch</th>
                          <th style="text-align: left;width: 15%;border-bottom: thin solid;color:green;">New Branch</th>
                        </tr>
                        <tr style="color:#008000;">
                          <td style="border-bottom: thin solid;">Positive</td>
                          <td style="border-bottom: thin solid;">[so+]</td>
                          <td style="border-bottom: thin solid;">[sn+]</td>
                          <td style="border-bottom: thin solid;">[f+]</td>
                          <td style="border-bottom: thin solid;">[o+]</td>
                          <td style="border-bottom: thin solid;">[n+]</td>
                        </tr>
                        <tr style="color:#FF0000;">
                          <td style="border-bottom: thin solid;">Negative</td>
                          <td style="border-bottom: thin solid;">[so-]</td>
                          <td style="border-bottom: thin solid;">[sn-]</td>
                          <td style="border-bottom: thin solid;">[f-]</td>
                          <td style="border-bottom: thin solid;">[o-]</td>
                          <td style="border-bottom: thin solid;">[n-]</td>
                        </tr>
                        <tr style="color:#808080;">
                          <td style="border-bottom: thin solid;">Neutre</td>
                          <td style="border-bottom: thin solid;">[so=]</td>
                          <td style="border-bottom: thin solid;">[sn=]</td>
                          <td style="border-bottom: thin solid;">[f=]</td>
                          <td style="border-bottom: thin solid;">[o=]</td>
                          <td style="border-bottom: thin solid;">[n=]</td>
                        </tr>
                        <tr style="color:#000000;">
                          <td style="border-bottom: thin solid;">Compouned</td>
                          <td style="border-bottom: thin solid;">[sox]</td>
                          <td style="border-bottom: thin solid;">[snx]</td>
                          <td style="border-bottom: thin solid;">[fx]</td>
                          <td style="border-bottom: thin solid;">[ox]</td>
                          <td style="border-bottom: thin solid;">[nx]</td>
                        </tr>
                      </table>
                 </div>
              """
    dNplBloc = dNplBloc.replace('[d0]', sda0)
    dNplBloc = dNplBloc.replace('[d1]', sda1)
    

    dNplBloc = dNplBloc.replace('[so+]', str(nplScore0[0]))
    dNplBloc = dNplBloc.replace('[so-]', str(nplScore0[1]))
    dNplBloc = dNplBloc.replace('[so=]', str(nplScore0[2]))
    dNplBloc = dNplBloc.replace('[sox]', str(nplScore0[3]))
    
    dNplBloc = dNplBloc.replace('[sn+]', str(nplScore1[0]))
    dNplBloc = dNplBloc.replace('[sn-]', str(nplScore1[1]))
    dNplBloc = dNplBloc.replace('[sn=]', str(nplScore1[2]))
    dNplBloc = dNplBloc.replace('[snx]', str(nplScore1[3]))
    
    fixscore = calculateNPL(fix)
    dNplBloc = dNplBloc.replace('[f+]', str(fixscore["pos"]))
    dNplBloc = dNplBloc.replace('[f-]', str(fixscore["neg"]))
    dNplBloc = dNplBloc.replace('[f=]', str(fixscore["neu"]))
    dNplBloc = dNplBloc.replace('[fx]', str(fixscore["compound"]))
    
    newscore = calculateNPL(new)
    dNplBloc = dNplBloc.replace('[n+]', str(newscore["pos"]))
    dNplBloc = dNplBloc.replace('[n-]', str(newscore["neg"]))
    dNplBloc = dNplBloc.replace('[n=]', str(newscore["neu"]))
    dNplBloc = dNplBloc.replace('[nx]', str(newscore["compound"]))
    
    oldscore = calculateNPL(old)
    dNplBloc = dNplBloc.replace('[o+]', str(oldscore["pos"]))
    dNplBloc = dNplBloc.replace('[o-]', str(oldscore["neg"]))
    dNplBloc = dNplBloc.replace('[o=]', str(oldscore["neu"]))
    dNplBloc = dNplBloc.replace('[ox]', str(oldscore["compound"]))
    
    return dNplBloc


def calculateDeltaArticles(sdate0, sdate1):
    a0 =  extractArticle(sdate0)
    a1 =  extractArticle(sdate1)
    wordproxies0, npl0 = splitArticle(a0)
    wordproxies1, npl1 = splitArticle(a1)
    nplScore0 = parseSimpleNPL(npl0)
    nplScore1 = parseSimpleNPL(npl1)
    res, fix, new, old = processSequences(wordproxies0, wordproxies1)
    dNplBloc = createDeltaNPL(fix, new, old, sdate0, sdate1, nplScore0, nplScore1)
    return res + '<br>' + dNplBloc


def generateNPLGraph():
    # initialize list of lists
    tsScore = []
    dateRegex = re.compile(r'(\d\d\d\d\d\d\d\d)')
    for datedFile in os.listdir('fedStates/'):
        if dateRegex.search(datedFile):
            a =  extractArticle(datedFile)
            w, npl = splitArticle(a)
            score = [float(s) for s in parseSimpleNPL(npl)] 
            score.insert(0, datedFile[0:4] + '-' + datedFile[4:6] + '-' + datedFile[6:8])
            tsScore.append(score)
    colors = ['green', 'red', 'blue', 'gray']
    df = pd.DataFrame(tsScore, columns =['Date', 'Positive', 'Negative', 'Neutre', 'Compuned'])
    df = df.sort_values(by=['Date'])
    df = df.melt('Date', var_name='NPL',  value_name='score')
    sns.set_palette(sns.color_palette(colors))
    sns_plot = sns.factorplot(x="Date", y="score", hue='NPL', data=df)
    sns_plot.set_xticklabels(rotation=45) 
    sns_plot.fig.set_figwidth(24)
    sns_plot.fig.set_figheight(8)
    sns_plot.savefig("./static/historicNpl.png")
    